## Latent Semantic Analysis

We will demonstrate LSA on a dataset available in the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). The [Reuters-21578 Text Categorization Collection Data Set](https://archive.ics.uci.edu/ml/datasets/Reuters-21578+Text+Categorization+Collection) contains a collection of documents that appeared on Reuters newswire in 1987. It is one of the widely used document collections for text categorization <cite data-cite="reuters21578"></cite>.

According to the description, the data is stored in 22 files in the SGML format, which is a superset of HTML and XML. Using the `BeautifulSoup4` package we will extract the data from SGML files. Then, `NLTK` and `Gensim` will be used for text processing. The following Python libraries are needed to run this notebook:

- gensim==3.8.3
- nltk==3.5
- beautifulsoup4==4.9.1

We will begin by downloading the archive and extracting `.sgm` files.

In [1]:
from urllib.request import urlopen
import tarfile

# download
data = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz')
with open('data/reuters21578.tar.gz', "wb") as fp:
    fp.write(data.read())
# read
files = []
tar = tarfile.open('data/reuters21578.tar.gz', 'r:gz')
for member in tar.getmembers():
    if member.name.endswith('.sgm'):
        files.append(tar.extractfile(member))

With the file objects now stored in `files`, we can proceed to extract the texts from SGML. We will configure the BeautifulSoup library to use the default HTML parser.

The SGML files are structured in such a way that each document is inside a `<TEXT>` element which contains sub-elements such as `<TITLE>`, `<AUTHOR>`, `<BODY>` etc. We will extract the `<TITLE>` and `<BODY>` elements for articles that do not have the `TYPE="BRIEF"` or `TYPE="UNPROC"` as these two contain only the title or are unusual in some fashion.

In [2]:
from bs4 import BeautifulSoup

documents = []
for f in files:
    sgml = BeautifulSoup(f, 'html.parser')
    texts = sgml.findAll('text')
    # filter types
    texts = [x for x in texts if x.attrs.get('type') not in ['BRIEF', 'UNPROC']]
    # find title and body and extract text
    texts = [(x.find('title').get_text(), x.find('body').get_text()) for x in texts]
    # end the title with a full stop and add the body
    texts = ['{}. {}'.format(title, body) for title, body in texts]
    # strip start-of-text and end-of-text ASCII characters and replace newlines with spaces
    texts = [x.strip('\x02\x03').replace('\n', ' ') for x in texts]
    documents.extend(texts)
print(len(documents))

19043


An inspection of the first 300 characters of the first document gives this result:

In [3]:
print(documents[0][:300])

BAHIA COCOA REVIEW. Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review.     The dry period means


The next step is to perform some basic text preprocessing. First, three NLTK packages need to be downloaded. Then, the documents will be tokenized using the NLTK's `word_tokenize` function which internally performs sentence splitting followed by tokenization.

In [4]:
import nltk
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
tokenized_documents = [nltk.tokenize.word_tokenize(doc) for doc in documents]

With lemmatization we will convert each word into its base form. For this task we will use the NLTK's Wordnet lemmatizer and provide part-of-speech (POS) tag of each word for better accuracy. The Wordnet lemmatizer uses Wordnet tags so we have to map POS tags to Wordnet tags first. Adjectives, verbs and adverbs are treated separately while everything else will be lemmatized as a noun.

In [5]:
import collections
# mapping from POS tags to Wordnet tags
tag_map = collections.defaultdict(lambda: nltk.corpus.wordnet.NOUN)
tag_map['J'] = nltk.corpus.wordnet.ADJ
tag_map['V'] = nltk.corpus.wordnet.VERB
tag_map['R'] = nltk.corpus.wordnet.ADV

lemmatized_documents = []
lemmatizer = nltk.stem.WordNetLemmatizer()
for doc_tokens in tokenized_documents:
    lemmatized_doc_tokens = [lemmatizer.lemmatize(token.lower(), tag_map[tag[0]]) for token, tag in nltk.pos_tag(doc_tokens)]
    lemmatized_documents.append(lemmatized_doc_tokens)

An inspection of the first 50 tokens of the lemmatized first documents gives this result:

In [6]:
print(lemmatized_documents[0][:50])

['bahia', 'cocoa', 'review', '.', 'shower', 'continue', 'throughout', 'the', 'week', 'in', 'the', 'bahia', 'cocoa', 'zone', ',', 'alleviate', 'the', 'drought', 'since', 'early', 'january', 'and', 'improve', 'prospect', 'for', 'the', 'come', 'temporao', ',', 'although', 'normal', 'humidity', 'level', 'have', 'not', 'be', 'restore', ',', 'comissaria', 'smith', 'say', 'in', 'it', 'weekly', 'review', '.', 'the', 'dry', 'period', 'mean']


Computing the average length of the documents returns:

In [7]:
sum([len(doc) for doc in lemmatized_documents])/len(lemmatized_documents)

158.90395420889567

The next step will be to remove tokens that are either too short, too long, composed of non-alphabetic characters or are stopwords (words that carry little or no meaning). We will use a standard stopword list available in the Gensim library. We will not develop our own corpus-specific list because it was demonstrated that this offers relatively little utility when training topic models <cite data-cite="stopwordsTopicModels"></cite>.

In [8]:
import gensim
lemmatized_documents = [[token for token in doc_tokens if 2<=len(token)<=15 and token.isalpha()] for doc_tokens in lemmatized_documents]
lemmatized_documents = [[token for token in doc_tokens if token not in gensim.parsing.preprocessing.STOPWORDS] for doc_tokens in lemmatized_documents]

The filtering reduced the average length of the documents:

In [9]:
sum([len(doc) for doc in lemmatized_documents])/len(lemmatized_documents)

74.39468571128499

The basic preprocessing is now complete. We proceed with the construction of the term-document matrix, a sparse matrix which contains document vectors. Different local and global weighting functions can be applied to transform each cell to be a product of a local term weight and a global weight. This is useful because rare words are likely to be more important than common words which appear in almost all documents. Raw term counts and TF-IDF weighting are two of the most popular combinations. We will use the Gensim library to perform _tf-idf_ weighting prior to LSA.

In [10]:
dic = gensim.corpora.Dictionary(lemmatized_documents)
bows = [dic.doc2bow(doc) for doc in lemmatized_documents]
tfidf_model = gensim.models.tfidfmodel.TfidfModel(dictionary=dic)
tfidf_vectors = tfidf_model[bows]

Using the computed tf-idf vectors we can proceed with LSA. Note that in information retrieval literature and computer programming libraries the terms LSA and LSI (latent semantic indexing) are often used interchangeably. However, LSI typically refers to the actual SVD decomposition and dimensionality reduction, while LSA denotes the application of the technique to a wider range of problems. The Gensim library implements this decomposition in its `LsiModel` class. The following snippet performs truncated SVD decomposition while keeping only `k=10` largest singular values.

In [11]:
lsi_model = gensim.models.LsiModel(corpus=tfidf_vectors, id2word=dic, num_topics=10)

Let's inspect the results of truncated SVD decomposition $X=U*S*V^T$: left singular vectors (U), singular values (S), and right singular vectors (V). The rows of the truncated matrix U are embeddings of words from the dictionary, while the columns of $V^T$ are embeddings of documents into the lower dimensional space.

In [12]:
import numpy as np
# human readable output of numpy matrices
np.set_printoptions(precision=4, suppress=True)
print(lsi_model.projection.u.shape)
print(lsi_model.projection.u)

(35177, 10)
[[-0.0001 -0.0011  0.0001 ...  0.0008  0.     -0.0001]
 [-0.0019 -0.0161  0.0022 ... -0.0099  0.0074 -0.0018]
 [-0.001  -0.0114  0.0017 ...  0.0093  0.0036 -0.0031]
 ...
 [-0.     -0.0001 -0.     ... -0.     -0.0001 -0.    ]
 [-0.     -0.0001 -0.     ... -0.     -0.0001 -0.    ]
 [-0.     -0.0001 -0.     ... -0.     -0.0001 -0.    ]]


In [13]:
np.set_printoptions(precision=3, suppress=True)
print(lsi_model.projection.s.shape)
print(lsi_model.projection.s)

(10,)
[26.138 17.003 13.266 11.915 10.412 10.096  9.71   9.231  9.042  8.748]


The matrix with right singular vectors can grow big with a large number of documents, therefore it is not stored with the model and we have to compute it explicitly:

In [14]:
C = gensim.matutils.corpus2dense(lsi_model[tfidf_vectors], len(lsi_model.projection.s)).T / lsi_model.projection.s
print(C.shape)
print(C)

(19043, 10)
[[-0.002 -0.008  0.001 ... -0.007  0.003 -0.   ]
 [-0.001 -0.007  0.001 ... -0.002  0.003 -0.001]
 [-0.001 -0.007  0.001 ...  0.011  0.004 -0.003]
 ...
 [-0.    -0.003  0.    ...  0.     0.001 -0.001]
 [-0.001 -0.003  0.001 ...  0.001 -0.001  0.   ]
 [-0.001 -0.006 -0.    ... -0.001 -0.003 -0.001]]


Now let's inspect how latent dimensions (left singular vectors or topics) are defined. We will use the `show_topics` functions which displays topics ordered by significance using the desired number of most important words.

In [15]:
lsi_model.show_topics(num_words=3)

[(0, '-0.432*"ct" + -0.375*"vs" + -0.371*"net"'),
 (1, '-0.241*"billion" + -0.236*"pct" + -0.220*"bank"'),
 (2, '0.471*"loss" + -0.371*"ct" + -0.338*"qtly"'),
 (3, '0.641*"loss" + -0.308*"net" + -0.243*"mln"'),
 (4, '0.363*"billion" + -0.337*"share" + -0.211*"offering"'),
 (5, '0.386*"billion" + -0.240*"tonne" + 0.199*"pct"'),
 (6, '-0.389*"bank" + -0.291*"stg" + 0.215*"billion"'),
 (7, '-0.346*"tonne" + 0.345*"billion" + -0.307*"stg"'),
 (8, '0.530*"stg" + -0.264*"bond" + 0.221*"mln"'),
 (9, '0.858*"oper" + -0.154*"billion" + 0.149*"excludes"')]

The topics may look cryptic at the first glance for several reasons. First and foremost, when using LSA, there is no straightforward way to interpret a topic defined in terms of linear combinations of words. Only in the most trivial and obvious cases can the positive and negative contributions of words be explained in a reasonable way. Second, the Reuters-21578 dataset contains many financial news and plenty of abbreviations like `ct`, `qtly`, etc. which are important when defining latent dimensions in financial domains but are hard to interpret without background knowledge about the data. Finally, our _apriori_ decision that we keep only the ten largest singular values (to keep things simple and outputs brief) is not appropriate for a dataset of this size and structure. Further experiments by the reader, using different parameters, could improve the results (typical settings in the literature range from 100 to a few thousand in case of hundreds of thousands of documents).

## Latent Dirichlet allocation

Latent Dirichlet allocation is very similar to probabilistic LSA (pLSA) but the topic distribution is assumed to have a sparse Dirichlet prior. To demonstrate LDA, we will use the same Reuters-21578 dataset and the same preprocessing that was used for LSA. Assuming that we have the `lemmatized_documents`, `dic`, and `bows` variables still populated, we can use the Gensim's `LdaModel` class.

We will train a LDA model with 10 latent topics. Next, we will inspects the topics and their most relevant words.

In [16]:
lda = gensim.models.LdaModel(bows, id2word=dic, num_topics=10)
lda.show_topics(num_words=4)

[(0, '0.017*"dlrs" + 0.013*"reuter" + 0.009*"new" + 0.008*"tonne"'),
 (1, '0.030*"dlrs" + 0.028*"pct" + 0.025*"mln" + 0.024*"company"'),
 (2, '0.036*"bank" + 0.021*"mln" + 0.020*"dlrs" + 0.018*"debt"'),
 (3, '0.048*"oil" + 0.019*"pct" + 0.016*"share" + 0.015*"company"'),
 (4, '0.026*"price" + 0.023*"president" + 0.016*"reuter" + 0.013*"executive"'),
 (5, '0.015*"billion" + 0.015*"market" + 0.015*"pct" + 0.013*"rate"'),
 (6, '0.015*"bond" + 0.013*"reuter" + 0.013*"issue" + 0.012*"pct"'),
 (7, '0.049*"share" + 0.033*"stock" + 0.032*"company" + 0.026*"reuter"'),
 (8, '0.019*"trade" + 0.014*"ec" + 0.013*"tonne" + 0.012*"year"'),
 (9, '0.130*"mln" + 0.084*"ct" + 0.078*"net" + 0.058*"vs"')]

The computed topics are too general to get meaningful insights, so we will repeat the training with 100 topics and display 10 of them.

In [17]:
lda = gensim.models.LdaModel(bows, id2word=dic, num_topics=100)
lda.show_topics(num_topics=5, num_words=4)

[(15,
  '0.056*"texaco" + 0.037*"canadian" + 0.036*"canada" + 0.029*"philippine"'),
 (44,
  '0.038*"american" + 0.031*"express" + 0.029*"company" + 0.022*"shearson"'),
 (20, '0.026*"brand" + 0.024*"port" + 0.020*"uganda" + 0.019*"dlrs"'),
 (58,
  '0.095*"telephone" + 0.080*"british" + 0.042*"aerospace" + 0.037*"week"'),
 (70, '0.098*"pacific" + 0.067*"canadian" + 0.048*"airline" + 0.045*"air"')]

It is also possible to get the most relevant topics for a given word to obtain its sparse vector representation:

In [18]:
lda.get_term_topics(dic.token2id['reagan'])

[(55, 0.016039087), (61, 0.091784805)]

The distribution of topics for the given document can be obtained (a sparse vector representation of the document). The following snippet displays the probability of the three strongest topics for the first document along with the five strongest topic words for each topic.

In [19]:
top_3 = sorted(lda.get_document_topics(bows[0]), key=lambda x: x[1], reverse=True)[:3]
for topic_id, p in top_3:
    print(p, [x[0] for x in lda.show_topic(topic_id)][:5])   

0.39589494 ['tonne', 'export', 'mln', 'wheat', 'sugar']
0.11983482 ['kong', 'hong', 'new', 'dlrs', 'philip']
0.087170556 ['coffee', 'quota', 'producer', 'cocoa', 'consumer']
